In [1]:
import os
from camel.configs import QwenConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.agents import ChatAgent
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from camel.loaders import Firecrawl
import qrcode
from PyPDF2 import PdfReader
os.environ["QWEN_API_KEY"] = "your_qwen_api_key"
os.environ["FIRECRAWL_API_KEY"] = "your_firecrawl_api_key"
secret_id = "your_secret_id"
secret_key = "your_secret_key"
region = 'ap-shanghai'
qwen_model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_TURBO,
    model_config_dict=QwenConfig(temperature=0.2).as_dict(),
)

2024-12-22 14:40:48,901 - camel - INFO - Camel library logging has been configured.


In [2]:
import random
import string

def generate_random_filename(length=10):
    """生成一个随机文件名字符串，默认长度为10个字符"""
    letters = string.ascii_letters + string.digits  # 包含字母和数字
    random_filename = ''.join(random.choice(letters) for i in range(length))
    return random_filename

# 示例用法
print(generate_random_filename())

jt4f24ezk7


In [ ]:
firecrawl = Firecrawl()
url_agent = ChatAgent(
    system_message="You're a helpful assistant",  # Define the agent's role or purpose
    message_window_size=10,  # [Optional] Specifies the chat memory length
    model=qwen_model,
    output_language="Chinese"
)

config = CosConfig(Region=region, SecretId=secret_id, SecretKey=secret_key)
client = CosS3Client(config)
def generate_qr_code(url):
    # 生成二维码
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(url)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img_path = generate_random_filename()+".png"
    img.save(img_path)  # 保存二维码为图片文件
    bucket_name = "idealeap-1254110372"
    # 上传文件
    with open(img_path, 'rb') as file:
        response = client.put_object(
            Bucket=bucket_name,
            Body=file,
            Key=f'Other/{img_path}'  # 上传到 COS 的文件名
        )

    # 获取文件的 URL
    url = f"https://{bucket_name}.cos.{region}.myqcloud.com/Other/{img_path}"  # 生成文件的 URL
    os.remove(img_path)  # 上传后删除本地文件
    print("文件的 URL",url)

    return url

def process_url(url):

    url_agent.reset()
    # Use the `crawl` method to retrieve content from the specified URL
    firecrawl_response = firecrawl.crawl(
        url=url  # Target URL to crawl for content
    )
    #print(firecrawl_response)
    # Use the ChatAgent to generate a response based on the Firecrawl crawl data
    response = url_agent.step(f"Based on {firecrawl_response}, help me generate a title ,summarize the full text.The word count should not exceed 20 words.Chinese")
    return response.msgs[0].content

def process_pdf(url):
    reader = PdfReader(url)
    pdf_text = ""
    for page in reader.pages:
        pdf_text += page.extract_text()
    pdf_text
    url_agent.reset()
    response = url_agent.step(f"Based on {pdf_text}, help me generate a title ,summarize the full text.The word count should not exceed 20 words.Chinese")
    return response.msgs[0].content


2024-12-22 14:40:50,371 - qcloud_cos.cos_client - INFO - generate built-in connection pool success. maxsize=10,10
2024-12-22 14:40:50,372 - qcloud_cos.cos_client - INFO - bound built-in connection pool when new client. maxsize=10,10


In [4]:
code_sys_msg = """# 角色
你是一位专业的前端设计大师，擅长运用精湛的 html+tailwind 技术，根据用户提供的文本内容和图片 url 链接，高效地生成用于在社交媒体或其他平台上分享内容的精美视觉化卡片。

## 技能
### 技能 1：生成文本卡片
1. 当用户提供文本内容时，将文本进行排版优化，使其在卡片上呈现出清晰、美观的效果。
2. 根据文本内容的主题和风格，选择合适的字体、颜色和布局。

### 技能 2：处理图片
1. 当用户提供图片 URL 链接时，确保图片能够正确加载并在卡片上显示。
2. 根据图片的尺寸和比例，调整卡片的布局，使图片和文本内容协调搭配。

## 示例
### 示例 1：生成带背景图卡片
1. 当用户提供标题、描述、图片 url 和二维码 url 时，按照带背景图卡片的格式生成 html+tailwind 代码。
2. 确保图片标签的 class 样式设置宽高，使图片在卡片中呈现出最佳效果。有图片，外层容器不设置宽高，防止图片过大挤满全屏！
===回复示例===
<div class="rounded-2xl overflow-hidden flex flex-col">
  <img src="用户提供的图片 url" class="w-95 h-30 object-cover"/>
<div class="flex px-5 py-4"> <div class="flex-col flex mr-2">
    <div class="text-2xl font-bold text-gray-900">用户提供的标题</div>
    <p class="text-gray-600 text-sm font-sans">用户提供的描述</p>
  </div><img src="用户提供的二维码 url" class="h-20 w-20 object-cover -mt-1"/></div>
</div>
===示例结束===

### 示例 2：生成无图片链接卡片
1. 当用户仅提供标题和描述时，按照无图片链接卡片的格式生成 html+tailwind 代码。
2. 卡片宽高限制在 w-72 h-50或者h-40（有图片则50，没有则40）
===回复示例===
<div class="rounded-2xl overflow-hidden w-72 h-40 flex flex-col">
  <div class="flex px-5 py-4">
    <div class="flex-col flex-1 mr-2">
      <div class="text-2xl font-bold text-gray-900 mb-2 font-sans">用户提供的标题</div>
      <p class="text-gray-600 text-sm">用户提供的描述</p>
    </div>
  </div>
</div>
===示例结束===

### 示例 3：生成二维码卡片
1. 当用户仅提供标题、描述、二维码时，按照二维码卡片的格式生成 html+tailwind 代码。
===回复示例===
<div class="rounded-2xl overflow-hidden w-95 h-32 flex flex-col">
  <div class="flex px-5 py-4">
    <div class="flex-col flex-1 mr-2">
      <div class="text-2xl font-bold text-gray-900 font-sans">用户提供的标题</div>
      <p class="text-gray-600 text-sm">用户提供的描述</p>
    </div>
    <img src="用户提供的二维码 url" class="h-20 w-20 object-cover mt-1"/>
  </div>
</div>
===示例结束===

## 限制:
1. 图片 class 样式必须设置宽高，若无图片链接则不写图片标签。
2. 样式统一使用 tailwind 方式写，含有子内容的父标签必须添加'flex'样式。
3. 禁止输出注释等无关代码。

## 输出要求
输出以"<div"开始，"</div>"结束"""
improve_sys_msg = """你是一个前端设计专家，对于前端给出代码，能够更加细致的调整样式，应用塔格式理论，让分享卡片更美观。
优化方向：
1. 层级尽量简洁，不过度用div包裹
2. 样式统一使用tailwind方式写，含有子内容的父标签必须添加'flex'样式
3. 样式布局更加合理，不使用flex-1。
4. 元素宽高不超过容器最大宽高420*360
5. 图片若没有链接则删除，否则必须设置具体宽高，不能设置w-full

请务必遵循输出要求！输出以```html开始"""
struct_sys_msg = """# 代码格式校验专家
根据html+tailwind前端代码，检查是否符合要求，若符合则原样输出，不符合修改后输出

## 限制:
1. 只根据用户提供的文本内容和图片 URL 链接进行卡片生成，不使用其他未经授权的素材。
2. 样式统一使用 tailwind 方式写，含有子内容的父标签必须添加'flex'样式。
4. 禁止输出注释等无关代码。

## 输出要求
输出以"<div"开始，"</div>"结束"""


In [5]:

code_agent = ChatAgent(
    system_message=code_sys_msg,
    model=qwen_model,
    message_window_size=10, # [Optional] the length for chat memory
    )
improve_agent = ChatAgent(
    system_message=improve_sys_msg,
    model=qwen_model,
    message_window_size=10, # [Optional] the length for chat memory
    )
struct_agent = ChatAgent(
    system_message=struct_sys_msg,
    model=qwen_model,
    message_window_size=10, # [Optional] the length for chat memory
    )

In [6]:
import requests
import json
from IPython.display import SVG, display
import re

def extract_html_content(text):
    """
    判断字符串是否包含 ```html ... ``` 代码块，并提取其中的内容。

    参数:
        text (str): 要检查的字符串。

    返回:
        str or None: 如果找到 ```html ... ``` 代码块，返回其中的内容；否则返回 None。
    """
    # 定义正则表达式模式
    pattern = r'```html\s*([\s\S]*?)```'

    # 使用 re.search 查找匹配
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)

    if match:
        # 提取并返回内容
        return match.group(1).strip()
    else:
        return text
def generate_svg(code):
    url = "https://ai.idealeap.cn/generate-svg"
    
    payload = json.dumps({
        "code": extract_html_content(code)
    })
    headers = {
        'Content-Type': 'application/json'
    }
    
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()  # 检查请求是否成功
        print("test",code,response.text)
        svg_data = response.text
        # display(SVG(data=response.text))
        return svg_data
    except requests.exceptions.HTTPError as http_err:
        return f"HTTP 错误发生: {http_err}"
    except Exception as err:
        return f"其他错误发生: {err}"

In [7]:
# Define a user message
# usr_msg = """用户输入是“习近平视察澳门：擘画新发展共创辉煌
# 习近平主席视察澳门，强调“一国两制”成功实践，鼓励澳门抓住发展机遇，共创辉煌。 没有图片链接”\n"""

# # Sending the message to the agent
# response = agent.step(usr_msg)

# # Check the response (just for illustrative purpose)
# print(response.msgs[0].content)
ouput = ''
def process_message(input):
    code_agent.reset()
    response = code_agent.step(input)
    ouput = response.msgs[0].content
    generate_svg(ouput)
    struct_agent.reset()
    response = struct_agent.step(input+ouput)
    ouput = response.msgs[0].content
    svg_data = generate_svg(ouput)
    return svg_data
# process_message(usr_msg)


In [8]:
import gradio as gr
from fish_audio_sdk import Session, ASRRequest

# 初始化 fish_audio_sdk 会话
API_KEY = "bfc2fade0a5e4e3cad9842045a72a73d"  # 替换为你的实际 API 密钥
session = Session(API_KEY)

def process_input(text_input, audio_input,url_input, file_input):
    """
    处理文本和音频输入，生成 SVG 卡片。

    参数:
    - text_input (str): 用户输入的文本。
    - audio_input (str): 上传的音频文件的路径。

    返回:
    - str: 生成的 SVG 卡片的 HTML 内容。
    """
    if file_input is not None and file_input != "":
        try:
            svg_output = process_message(process_pdf(file_input))
            return svg_output
        except Exception as e:
            print(f"URL 处理错误: {e}")
            return "<p style='color:red;'>URL 处理失败，请检查 URL 是否有效。</p>"
    if url_input is not None and url_input != "":
        try:
            svg_output = process_message(process_url(url_input)+'\n二维码图片链接：'+generate_qr_code(url_input))
            return svg_output
        except Exception as e:
            print(f"URL 处理错误: {e}")
            return "<p style='color:red;'>URL 处理失败，请检查 URL 是否有效。</p>"
    elif audio_input is not None:
        try:
            file_path = audio_input
            print(f"音频文件路径: {file_path}",audio_input)
            # 获取当前时间，用于生成唯一文件名
            filename = "audio.mp3"
            save_path = os.path.join('', filename)
            
            # 将临时文件复制到目标路径
            with open(file_path, "rb") as src, open(save_path, "wb") as dst:
                dst.write(src.read())
            # 读取音频文件的二进制数据
            with open(file_path, "rb") as audio_file:
                audio_data = audio_file.read()
            
            # 使用 fish_audio_sdk 进行语音转文字
            asr_request = ASRRequest(audio=audio_data, ignore_timestamps=True)
            response = session.asr(asr_request)
            
            transcribed_text = response.text
            print(f"转录的文本: {transcribed_text}")
        except Exception as e:
            print(f"音频处理错误: {e}")
            return "<p style='color:red;'>音频处理失败，请检查音频文件格式或内容。</p>"
    elif text_input:
        transcribed_text = text_input
    else:
        return "<p style='color:red;'>请提供文本或上传音频。</p>"
    
    try:
        # 调用现有的 process_message 函数生成 SVG
        svg_output = process_message(transcribed_text)
        return svg_output
    except Exception as e:
        print(f"SVG 生成错误: {e}")
        return "<p style='color:red;'>生成 SVG 卡片时发生错误。</p>"

/Users/marlene/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import gradio as gr

input_audio = gr.Audio(
    sources="microphone",
    type="filepath",
    label="录制音频",
    # waveform_options=gr.WaveformOptions(
    #     waveform_color="#01C6FF",
    #     waveform_progress_color="#0066B4",
    #     skip_length=2,
    #     show_controls=False,
    # ),
)
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Textbox(lines=10, placeholder="在这里输入你的文本...", label="输入文本"),
        input_audio,
        gr.Textbox(
            lines=1, placeholder="在这里输入你的网页链接...", label="输入网页链接"
        ),
        gr.Textbox(
            lines=1, placeholder="在这里输入你的本地pdf链接...", label="输入本地pdf链接"
        ),
    ],
    outputs=gr.HTML(label="输出 SVG 卡片"),
    title="卡片生成器",
    description="输入你的文本、链接或上传音频，生成对应的 SVG 卡片。",
)
iface.launch()

2024-12-22 14:40:52,686 - httpx - INFO - HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
* Running on local URL:  http://127.0.0.1:7860
2024-12-22 14:40:52,954 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2024-12-22 14:40:52,973 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"

To create a public link, set `share=True` in `launch()`.


2024-12-22 14:40:53,623 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2024-12-22 14:42:08,063 - httpx - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-22 14:42:08,078 - camel.agents.chat_agent - INFO - Model qwen-turbo, index 0, processed these messages: [{'role': 'system', 'content': '# 角色\n你是一位专业的前端设计大师，擅长运用精湛的 html+tailwind 技术，根据用户提供的文本内容和图片 url 链接，高效地生成用于在社交媒体或其他平台上分享内容的精美视觉化卡片。\n\n## 技能\n### 技能 1：生成文本卡片\n1. 当用户提供文本内容时，将文本进行排版优化，使其在卡片上呈现出清晰、美观的效果。\n2. 根据文本内容的主题和风格，选择合适的字体、颜色和布局。\n\n### 技能 2：处理图片\n1. 当用户提供图片 URL 链接时，确保图片能够正确加载并在卡片上显示。\n2. 根据图片的尺寸和比例，调整卡片的布局，使图片和文本内容协调搭配。\n\n## 示例\n### 示例 1：生成带背景图卡片\n1. 当用户提供标题、描述、图片 url 和二维码 url 时，按照带背景图卡片的格式生成 html+tailwind 代码。\n2. 确保图片标签的 class 样式设置宽高，使图片在卡片中呈现出最佳效果。有图片，外层容器不设置宽高，防止图片过大挤满全屏！\n===回复示例===\n<div class="rounded-2xl overflow-hidden flex flex-col">\n  <img src="用户提供的图片 url" class="w-95 h-30 object-cover"/>\n<div class="flex 